In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [5]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [154]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.27557887
====> Test set loss: 1.2546, 66.0%
====> Epoch: 150 Average loss: 1.24960347
====> Test set loss: 1.1862, 68.0%
====> Epoch: 225 Average loss: 1.21563738
====> Test set loss: 1.1894, 66.5%
====> Epoch: 300 Average loss: 1.19709698
====> Test set loss: 1.1760, 67.5%
====> Epoch: 375 Average loss: 1.21821563
====> Test set loss: 1.1765, 68.0%
====> Epoch: 450 Average loss: 1.19930835
====> Test set loss: 1.1765, 68.0%
====> Epoch: 525 Average loss: 1.16457643
====> Test set loss: 1.1754, 68.0%
====> Epoch: 600 Average loss: 1.17904321
====> Test set loss: 1.1766, 68.0%
====> Epoch: 675 Average loss: 1.17668282
====> Test set loss: 1.1764, 68.0%
====> Epoch: 750 Average loss: 1.24575864
====> Test set loss: 1.1767, 68.0%
Training state:  False
Elapsed:  50.90697503089905
Complete set accuracy: 72.89999999999999%


### Runner

In [9]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(374, 400):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 350
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.25568018
====> Test set loss: 1.1964, 72.0%
====> Epoch: 150 Average loss: 1.19291152
====> Test set loss: 1.2168, 71.0%
====> Epoch: 225 Average loss: 1.18362179
====> Test set loss: 1.2083, 71.5%
====> Epoch: 300 Average loss: 1.16191930
====> Test set loss: 1.2086, 71.5%
====> Epoch: 375 Average loss: 1.18155717
====> Test set loss: 1.2108, 72.0%
====> Epoch: 450 Average loss: 1.17700907
====> Test set loss: 1.2107, 72.0%
====> Epoch: 525 Average loss: 1.16835340
====> Test set loss: 1.2097, 72.0%
====> Epoch: 600 Average loss: 1.21786142
====> Test set loss: 1.2085, 72.0%
====> Epoch: 675 Average loss: 1.19034091
====> Test set loss: 1.2076, 72.0%
====> Epoch: 750 Average loss: 1.16330194
====> Test set loss: 1.2073, 72.0%
Training state:  False
Complete set accuracy: 74.7%
Log accuracy: 72.3%
---- Done in  48.380911111831665  seconds

---- Running for model name:  B_add_mild_nlin
=

====> Epoch: 675 Average loss: 1.24463137
====> Test set loss: 1.2772, 61.0%
====> Epoch: 750 Average loss: 1.24202299
====> Test set loss: 1.2770, 61.0%
Training state:  False
Complete set accuracy: 67.80000000000001%
Log accuracy: 67.80000000000001%
---- Done in  56.989131689071655  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.30739464
====> Test set loss: 1.2535, 69.5%
====> Epoch: 150 Average loss: 1.21024837
====> Test set loss: 1.1431, 71.0%
====> Epoch: 225 Average loss: 1.25763862
====> Test set loss: 1.1368, 71.0%
====> Epoch: 300 Average loss: 1.20029579
====> Test set loss: 1.1364, 71.0%
====> Epoch: 375 Average loss: 1.20933378
====> Test set loss: 1.1350, 71.0%
====> Epoch: 450 Average loss: 1.19990117
====> Test set loss: 1.1348, 70.5%
====> Epoch: 525 Average loss: 1.21574175
====> Test set loss: 1.1352, 70.0%
====> Epoch: 600 Average loss: 1.26337729
====> Test set loss: 1.1356, 70.0%
====> Epoch: 675 Average loss: 1.25780586
===

====> Epoch: 450 Average loss: 1.17696799
====> Test set loss: 1.1684, 71.0%
====> Epoch: 525 Average loss: 1.16631554
====> Test set loss: 1.1687, 71.0%
====> Epoch: 600 Average loss: 1.14963442
====> Test set loss: 1.1693, 71.0%
====> Epoch: 675 Average loss: 1.18514573
====> Test set loss: 1.1694, 71.0%
====> Epoch: 750 Average loss: 1.19131897
====> Test set loss: 1.1699, 71.0%
Training state:  False
Complete set accuracy: 75.1%
Log accuracy: 72.89999999999999%
---- Done in  56.20734691619873  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.20873743
====> Test set loss: 1.1508, 74.0%
====> Epoch: 150 Average loss: 1.16814780
====> Test set loss: 1.1453, 74.5%
====> Epoch: 225 Average loss: 1.18713185
====> Test set loss: 1.1345, 74.5%
====> Epoch: 300 Average loss: 1.14405288
====> Test set loss: 1.1309, 74.5%
====> Epoch: 375 Average loss: 1.20478289
====> Test set loss: 1.1320, 74.5%
====> Epoch: 450 Average loss: 1.16114347
====> Test

====> Epoch: 225 Average loss: 1.19895803
====> Test set loss: 1.1267, 75.0%
====> Epoch: 300 Average loss: 1.17889416
====> Test set loss: 1.1242, 76.0%
====> Epoch: 375 Average loss: 1.16123245
====> Test set loss: 1.1221, 76.0%
====> Epoch: 450 Average loss: 1.18303722
====> Test set loss: 1.1230, 75.5%
====> Epoch: 525 Average loss: 1.19651368
====> Test set loss: 1.1243, 76.0%
====> Epoch: 600 Average loss: 1.16445233
====> Test set loss: 1.1250, 76.0%
====> Epoch: 675 Average loss: 1.18368953
====> Test set loss: 1.1245, 76.0%
====> Epoch: 750 Average loss: 1.16929986
====> Test set loss: 1.1239, 76.0%
Training state:  False
Complete set accuracy: 74.1%
Log accuracy: 73.3%
---- Done in  60.009681940078735  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.27190783
====> Test set loss: 1.2002, 72.0%
====> Epoch: 150 Average loss: 1.21918998
====> Test set loss: 1.1250, 72.5%
====> Epoch: 225 Average loss: 1.19062016
====> Test set loss: 1.1

====> Epoch: 75 Average loss: 1.24006800
====> Test set loss: 1.2622, 66.0%
====> Epoch: 150 Average loss: 1.19815622
====> Test set loss: 1.2602, 68.0%
====> Epoch: 225 Average loss: 1.19025974
====> Test set loss: 1.2441, 70.0%
====> Epoch: 300 Average loss: 1.21326217
====> Test set loss: 1.2392, 71.0%
====> Epoch: 375 Average loss: 1.18600219
====> Test set loss: 1.2348, 70.0%
====> Epoch: 450 Average loss: 1.17144391
====> Test set loss: 1.2352, 70.0%
====> Epoch: 525 Average loss: 1.16516742
====> Test set loss: 1.2352, 70.0%
====> Epoch: 600 Average loss: 1.18690677
====> Test set loss: 1.2345, 70.0%
====> Epoch: 675 Average loss: 1.18590421
====> Test set loss: 1.2346, 70.0%
====> Epoch: 750 Average loss: 1.19455547
====> Test set loss: 1.2359, 70.0%
Training state:  False
Complete set accuracy: 73.6%
Log accuracy: 72.7%
---- Done in  58.25867199897766  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.30212216
====> Test set loss: 1.2614, 6

...

====> Epoch: 300 Average loss: 1.15253547
====> Test set loss: 1.1220, 70.5%
====> Epoch: 375 Average loss: 1.12695464
====> Test set loss: 1.1184, 70.5%
====> Epoch: 450 Average loss: 1.10906582
====> Test set loss: 1.1198, 70.5%
====> Epoch: 525 Average loss: 1.13797525
====> Test set loss: 1.1204, 70.5%
====> Epoch: 600 Average loss: 1.11120049
====> Test set loss: 1.1214, 70.5%
====> Epoch: 675 Average loss: 1.14321380
====> Test set loss: 1.1219, 70.5%
====> Epoch: 750 Average loss: 1.14059513
====> Test set loss: 1.1218, 70.5%
Training state:  False
Complete set accuracy: 75.6%
Log accuracy: 73.7%
---- Done in  64.99160885810852  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.27618444
====> Test set loss: 1.2080, 72.5%
====> Epoch: 150 Average loss: 1.21497431
====> Test set loss: 1.1668, 69.5%
====> Epoch: 225 Average loss: 1.18053310
====> Test set loss: 1.1697, 69.5%
====> Epoch: 300 Average loss: 1.15062432
====> Test set loss: 1.1708, 

====> Epoch: 75 Average loss: 1.28775082
====> Test set loss: 1.2262, 71.0%
====> Epoch: 150 Average loss: 1.19307925
====> Test set loss: 1.1443, 75.0%
====> Epoch: 225 Average loss: 1.24861538
====> Test set loss: 1.1353, 74.0%
====> Epoch: 300 Average loss: 1.24106548
====> Test set loss: 1.1308, 74.5%
====> Epoch: 375 Average loss: 1.25476869
====> Test set loss: 1.1263, 75.0%
====> Epoch: 450 Average loss: 1.20419068
====> Test set loss: 1.1265, 75.0%
====> Epoch: 525 Average loss: 1.22670428
====> Test set loss: 1.1261, 75.0%
====> Epoch: 600 Average loss: 1.21161679
====> Test set loss: 1.1259, 74.5%
====> Epoch: 675 Average loss: 1.21206807
====> Test set loss: 1.1258, 74.5%
====> Epoch: 750 Average loss: 1.23818992
====> Test set loss: 1.1259, 74.5%
Training state:  False
Complete set accuracy: 73.9%
Log accuracy: 67.9%
---- Done in  69.37506580352783  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.31018786
====> Test set loss: 1.1889, 7

====> Epoch: 750 Average loss: 1.15564517
====> Test set loss: 1.1960, 65.0%
Training state:  False
Complete set accuracy: 74.6%
Log accuracy: 73.1%
---- Done in  76.53437995910645  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.25277131
====> Test set loss: 1.1509, 73.5%
====> Epoch: 150 Average loss: 1.17457682
====> Test set loss: 1.0719, 75.5%
====> Epoch: 225 Average loss: 1.12975417
====> Test set loss: 1.0706, 75.5%
====> Epoch: 300 Average loss: 1.13469091
====> Test set loss: 1.0666, 76.0%
====> Epoch: 375 Average loss: 1.08823805
====> Test set loss: 1.0683, 76.0%
====> Epoch: 450 Average loss: 1.18832690
====> Test set loss: 1.0682, 76.0%
====> Epoch: 525 Average loss: 1.14056441
====> Test set loss: 1.0686, 76.0%
====> Epoch: 600 Average loss: 1.15558347
====> Test set loss: 1.0688, 76.0%
====> Epoch: 675 Average loss: 1.15248891
====> Test set loss: 1.0690, 76.0%
====> Epoch: 750 Average loss: 1.14897921
====> Test set loss: 1.

====> Epoch: 525 Average loss: 1.13711600
====> Test set loss: 1.0791, 73.0%
====> Epoch: 600 Average loss: 1.17770157
====> Test set loss: 1.0790, 73.0%
====> Epoch: 675 Average loss: 1.11932936
====> Test set loss: 1.0789, 73.0%
====> Epoch: 750 Average loss: 1.15226843
====> Test set loss: 1.0785, 73.0%
Training state:  False
Complete set accuracy: 77.10000000000001%
Log accuracy: 74.4%
---- Done in  69.36953806877136  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.27877927
====> Test set loss: 1.2525, 65.0%
====> Epoch: 150 Average loss: 1.22447498
====> Test set loss: 1.1796, 66.0%
====> Epoch: 225 Average loss: 1.19824264
====> Test set loss: 1.1761, 68.5%
====> Epoch: 300 Average loss: 1.20964701
====> Test set loss: 1.1752, 68.5%
====> Epoch: 375 Average loss: 1.19820640
====> Test set loss: 1.1804, 69.0%
====> Epoch: 450 Average loss: 1.15873300
====> Test set loss: 1.1794, 69.0%
====> Epoch: 525 Average loss: 1.19744670
====> Test s

KeyboardInterrupt: 